<a href="https://colab.research.google.com/github/amaithi-sam/rag-neo4j-groq/blob/main/RAG_with_Neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet langchain langchain-community langchain-groq langchain-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00


In [6]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=a3539f547a5a8307eea19d784343d62a0911f182dcff8e5364dd14ca06a00a77
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [31]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.9 MB/s eta 0:00:00


In [14]:
!pip install neo4j yfiles_jupyter gpt4all
!pip install neo4j

  Using cached neo4j-5.21.0-py3-none-any.whl (286 kB)
ERROR: Could not find a version that satisfies the requirement yfiles_jupyter (from versions: none)
ERROR: No matching distribution found for yfiles_jupyter
  Using cached neo4j-5.21.0-py3-none-any.whl (286 kB)


In [50]:


import os
os.environ["GROQ_API"] = GROQ_API
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD


In [15]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [9]:
from langchain.document_loaders import WikipediaLoader

raw_documents = WikipediaLoader(query="Elizabeth I").load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [10]:
raw_documents

[Document(page_content='Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.\nElizabeth was the only surviving child of Henry VIII and Anne Boleyn, his second wife, who was executed when Elizabeth was two years old. Anne\'s marriage to Henry was annulled, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry\'s death in 1547, Elizabeth\'s younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward\'s will was set aside within weeks of his death and Mary became queen, deposing and executing Jane. During Mary\'s reign, Elizabeth was imprisoned for nearly a year on suspic

document splitting

In [32]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [33]:
len(documents)

6

In [34]:
from langchain_groq import ChatGroq

llm = ChatGroq(temperature =0, groq_api_key=GROQ_API, model = 'llama3-70b-8192')

In [35]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(llm=llm)

In [36]:
llm.invoke('hello')

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_time': 0.07006282, 'prompt_time': 0.004319724, 'queue_time': None, 'total_time': 0.074382544}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_abd29e8833', 'finish_reason': 'stop', 'logprobs': None}, id='run-bf32fc4d-3a58-412d-b936-3b57b030b90b-0')

In [37]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)


In [38]:
graph_documents

[GraphDocument(nodes=[Node(id='Elizabeth I', type='Person'), Node(id='Henry Viii', type='Person'), Node(id='Anne Boleyn', type='Person'), Node(id='Edward Vi', type='Person'), Node(id='Lady Jane Grey', type='Person'), Node(id='Mary', type='Person'), Node(id='William Cecil', type='Person'), Node(id='James Vi Of Scotland', type='Person'), Node(id='Mary, Queen Of Scots', type='Person'), Node(id='Sir Francis Walsingham', type='Person')], relationships=[Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='Henry Viii', type='Person'), type='PARENT'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='Anne Boleyn', type='Person'), type='PARENT'), Relationship(source=Node(id='Henry Viii', type='Person'), target=Node(id='Edward Vi', type='Person'), type='PARENT'), Relationship(source=Node(id='Henry Viii', type='Person'), target=Node(id='Mary', type='Person'), type='PARENT'), Relationship(source=Node(id='Mary, Queen Of Scots', type='Person'), target=N

In [39]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel = True,
    include_source = True
)

In [40]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [43]:
%pip install --upgrade --quiet  gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 39.9 MB/s eta 0:00:00


In [44]:
from langchain_community.vectorstores import Neo4jVector

In [47]:
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}

embeddings = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
)

Downloading: 100%|██████████| 45.9M/45.9M [00:00<00:00, 59.7MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 559MiB/s]


In [51]:
from langchain_community.embeddings import GPT4AllEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [52]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [54]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


In [55]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [56]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [57]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [58]:
entity_chain.invoke({"question": "Where was Amelia Earhart born?"}).names

['Amelia Earhart']

In [59]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [60]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [61]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [62]:
print(structured_retriever("Who is Elizabeth I?"))

Elizabeth I - PARENT -> Henry Viii
Elizabeth I - PARENT -> Anne Boleyn
Elizabeth I - ADVISOR -> William Cecil
Elizabeth I - MINISTER -> Sir Francis Walsingham
Elizabeth I - SUCCESSION -> James Vi Of Scotland
Henry Viii - SUCCESSION -> Elizabeth I
Elizabeth - NAMESAKE -> Elizabeth Of York


In [63]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [64]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [65]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [66]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [69]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnablePassthrough,
    RunnableParallel
)
from langchain_core.output_parsers import StrOutputParser


_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | llm
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [70]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [71]:
prompt = ChatPromptTemplate.from_template(template)

In [72]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [75]:
from langchain_core.messages import AIMessage, HumanMessage
chain.invoke({"question": "Which house did Elizabeth I belong to?"})

Search query: Which house did Elizabeth I belong to?


'Elizabeth I belonged to the House of Tudor.'

In [76]:


chain.invoke(
    {
        "question": "When was she born?",
        "chat_history": [("Which house did Elizabeth I belong to?", "House Of Tudor")],
    }
)

Search query: When was Elizabeth I born?


'Elizabeth I was born on September 7, 1533.'